# Google Maps geocoder

In [1]:
import pandas as pd
import requests
import logging
import time
from creds import *

In [2]:
#request cache
key = my_api_key
API_KEY = my_api_key

In [3]:
logger = logging.getLogger("root")
logger.setLevel(logging.DEBUG)
logger.propagate = False

In [4]:
ch = logging.StreamHandler()
ch.setLevel(logging.DEBUG)
logger.addHandler(ch)

In [5]:
BACKOFF_TIME = 1
output_filename = '../apple-stores/data/processed/apple_store_locations_geocoded.csv'
input_filename = '../apple-stores/data/processed/apple_store_locations.csv'
address_column_name = 'address'
RETURN_FULL_RESULTS = True
data = pd.read_csv(input_filename, encoding='utf8')

In [6]:
if address_column_name not in data.columns:
    raise ValueError("Missing Address column in input data")

addresses = data[address_column_name].tolist()

def get_google_results(full_address, api_key=my_api_key, return_full_response=True):
    geocode_url = "https://maps.googleapis.com/maps/api/geocode/json?address={}".format(address)
    if api_key is not None:
        geocode_url = geocode_url + '&key={}'.format(api_key)
        
    results = requests.get(geocode_url)
    results = results.json()
    
    if len(results['results']) == 0:
        output = {
            "formatted_address" : None,
            "latitude": None,
            "longitude": None,
            "accuracy": None,
            "google_place_id": None,
            "type": None,
            "postcode": None
        }
    else:    
        answer = results['results'][0]
        output = {
            "formatted_address" : answer.get('formatted_address'),
            "latitude": answer.get('geometry').get('location').get('lat'),
            "longitude": answer.get('geometry').get('location').get('lng'),
            "accuracy": answer.get('geometry').get('location_type'),
            "google_place_id": answer.get("place_id"),
            "type": ",".join(answer.get('types')),
            "postcode": ",".join([x['long_name'] for x in answer.get('address_components') 
                                  if 'postal_code' in x.get('types')])
        }
         
    output['input_string'] = full_address
    output['number_of_results'] = len(results['results'])
    output['status'] = results.get('status')
    if return_full_response is True:
        output['response'] = results
    
    return output

results = []
for address in addresses:
    geocoded = False
    while geocoded is not True:
        try:
            geocode_result = get_google_results(address, my_api_key, return_full_response=RETURN_FULL_RESULTS)
        except Exception as e:
            logger.exception(e)
            logger.error("Major error with {}".format(address))
            logger.error("Skipping!")
            geocoded = True
            
        if geocode_result['status'] == 'OVER_QUERY_LIMIT':
            logger.info("Hit Query Limit! Backing off for a bit.")
            time.sleep(BACKOFF_TIME * 60) # sleep for 30 minutes
            geocoded = False
        else:
            if geocode_result['status'] != 'OK':
                logger.warning("Error geocoding {}: {}".format(address, geocode_result['status']))
            logger.debug("Geocoded: {}: {}".format(address, geocode_result['status']))
            results.append(geocode_result)           
            geocoded = True

    if len(results) % 500 == 0:
        logger.info("Completed {} of {} address".format(len(results), len(addresses)))
            
    if len(results) % 500 == 0:
        pd.DataFrame(results).to_csv("{}_bak".format(output_filename))

logger.info("Finished geocoding all addresses")
pd.DataFrame(results).to_csv(output_filename, encoding='utf8')

Geocoded: 2240 Q Street NE, Albuquerque, New Mexico 87110: OK
Geocoded: 65 Broadway, New Haven, Connecticut 06511: OK
Geocoded: 356 Greenwich Avenue, Greenwich, Connecticut 06830: OK
Geocoded: 5065 Main Street, Trumbull, Connecticut 06611: OK
Geocoded: Seven Backus Avenue, Danbury, Connecticut 06810: OK
Geocoded: 500 Westfarms Mall, Farmington, Connecticut 06032: OK
Geocoded: 401 Evergreen Walk, South Windsor, Connecticut 06074: OK
Geocoded: 100 N. Water Street, Norwalk, Connecticut 06854: OK
Geocoded: 4305 La Jolla Village Drive, San Diego, California 92122: OK
Geocoded: 2125 Chestnut Street, San Francisco, California 94123: OK
Geocoded: 432 West Hillcrest Drive, Thousand Oaks, California 91360: OK
Geocoded: 1415 Third Street Promenade, Santa Monica, California 90401: OK
Geocoded: 1301 Burlingame Avenue, Burlingame, California 94010: OK
Geocoded: 6600 Topanga Canyon Boulevard, Canoga Park, California 91303: OK
Geocoded: 379 Stanford Shopping Center, Palo Alto, California 94304: OK
Geo